## **3. Pondération statistique** (TF-IDF)  

https://stackoverflow.com/questions/46580932/calculate-tf-idf-using-sklearn-for-n-grams-in-python  
http://scikit-learn.sourceforge.net/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn-feature-extraction-text-tfidfvectorizer  
https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction


In [83]:
path = '/Users/camilledemers/Documents/05-transformation/'
acteur = 'inesss'

In [84]:
import pandas as pd 

file_path = path + acteur + '/' + acteur + '_terms.csv'
with open(file_path) as f:
    csv = pd.read_csv(f)

In [85]:
vocabulaire = list(csv['Terme'])

In [86]:
# Importer les données (fichiers .txt)
import os

def lire_corpus(acteur):
    path = '/Users/camilledemers/Documents/03-corpus/2-data/1-fr/' + acteur + '/'

    corpus =  [] 
    corpus_index = []

    # Change the directory
    os.chdir(path)

    for file in os.listdir()[:75]: # les 75 premières pages seulement 
        if file.endswith(".txt"):
          file_path = path + file
        
        with open(file_path, 'r', encoding = "UTF-8") as f:
            try:
                data = f.readlines()
                corpus.append(data[1].lower().strip('\n'))
            
            except:
                True

    return corpus

corpus = lire_corpus(acteur)

print("On a un corpus de {} documents".format(len(corpus)))

On a un corpus de 75 documents


In [87]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(vocabulary = vocabulaire, stop_words=None, ngram_range=(2,8))
tfs = tfidf.fit_transform(corpus)

In [88]:
features_names = tfidf.get_feature_names_out()
corpus_index = [corpus.index(n) for n in corpus]

import pandas as pd
df = pd.DataFrame(tfs.T.todense(), index=features_names, columns=corpus_index)

In [89]:
path = '/Users/camilledemers/Documents/04-filtrage/' + acteur + '/'
file_path = path + acteur + '_matrice-TFIDF.csv'
df.to_csv(file_path)

In [90]:
terms_weighted = []
rows, cols = tfs.nonzero()
for row, col in zip(rows,cols):
    terms_weighted.append([features_names[col], tfs[row,col]])

In [91]:
terms_weighted = pd.DataFrame(terms_weighted, columns=['Terme', 'TF-IDF'])
terms_weighted.sort_values(["TF-IDF"], 
                    axis=0,
                    ascending=[False], 
                    inplace=True)
terms_weighted.to_csv(path + acteur + '_weighting_TF-IDF.csv')

In [92]:
terms_weighted

,Terme,TF-IDF
2265,rencontre statutaire,0.836008
23323,couvre feu,0.787861
20715,sras cov,0.735389
10882,médicament biosimilaire,0.728743
6768,scientifique santé,0.693650
...,...,...
5016,usage optimal protocoles,0.001529
5015,usage optimal protocoles médicaux,0.001529
5014,usage optimal protocoles médicaux nationaux,0.001529
5013,optimal protocoles,0.001529
